# Задание 1. MDP (30%)

Когда Инокетений Вальдемарович Носочков приходит на работу, то с вероятностью 50% он хочет пойти в бар, с 10% – покушац, с 20% – спать. Если он ничего не хочет, то он продолжает работать. Когда Кеша пьет в Баре, то в 10% посещений он возвращается на работу и с вероятностью 30% идет спать, но в остальное время продолжает пить. Когда он просыпается, то с вероятностью 40% идет покушац и с вероятностью 60% идет в бар пить дальше. Если вдруг г-н Носочков поел, то с вероятностью 80% он начинает работать, а если не срослось с работой, то он начинает хотеть спать.

Определите вероятности, что наш герой прямо сейчас работает, пьет в баре, спит или ест, при условии, что если Инокетений чего-то хочет, то делает это.

In [11]:
import numpy as np
#let's define Nosochkov's policy
#working, sleeping, eating, drinking
M = np.array([[0.2, 0.2, 0.1, 0.5],
                [0, 0, 0.4, 0.6],
                [0.8, 0.2, 0, 0],
                [0.1, 0.3, 0, 0.6]])

answer = np.ones((4)) * 0.25 @ np.linalg.matrix_power(M, 1000)
print("Perfect Kesha's work-life balance formula:\n \
    working - %.2f, sleeping - %.2f, eating - %.2f, drinking - %.2f" % tuple(answer))

Perfect Kesha's work-life balance formula:
     working - 0.17, sleeping - 0.21, eating - 0.10, drinking - 0.52


# Задание 2. Tabular Q-learning  (30%)

Условие игры: <br>
Игра начинается с 1 очка. <br>Два игрока по очереди прибавляют очки от 1 до 9. <br>Побеждает тот, кто первым наберет 100 и более очков.

Пример: <br>В игре 87 очков. <br>Первый прибавляет 7 -> получается 94. <br>Второй прибавляет 7, получается 101 и второй побеждает.

In [54]:
class Game:
    """
    A class to represent the game
    """
    points = 1
    turn = 0 # player who made the previous move 
        
    def __init__(self):
        """
        Initializes a new game
        """
        self.points = 1
        self.turn = 2
        return

    def reset_game(self):
        """
        Resets the game to starting position
        """
        self.points = 1
        self.turn = 2
        return
    
    def make_move(self, points):
        """
        Player makes a move
        """
        if points < 1 or points > 9:
            print("Invalid points")
            return
        if self.is_game_over()[0]:
            return
        self.points += points
        self.turn = int(not(self.turn-1))+1
        return
    
    def is_game_over(self):
        """
        Finds out weather the current game is over or not.
        """
        if self.points >= 100:
            return True, self.turn, 1 # Game ended, winner, reward
        return False, self.turn, 0

In [55]:
from random import randrange
class Random_player:
    """
    Player class
    """
    def move(self, points):
        """
        Player makes a move.
        """
        move = randrange(9) + 1
        return move

In [56]:
game = Game()

In [57]:
rplayer = Random_player()

In [58]:
reward = 0
for j in range(100):
    game.reset_game()
    while game.is_game_over()[0] == False:
        game.make_move(rplayer.move(game.points))
    if game.turn == 1:
        reward += 1
print("Random player victory ratio: %d" % reward, "%")

Random player victory ratio: 54 %


## Победите 10/10 используя Tabular Q-learning.
Вы играете за 1го игрока, второй игрок ходит рандомно.

In [62]:
from collections import defaultdict

n = 9
Q = defaultdict(lambda: np.zeros(9))

In [63]:
def eps_greedy(Q, state, epsilon):
    if np.random.random() < epsilon:
        return np.random.choice(n)
    else:
        return np.argmax(Q[state])
    
def q_eval(Q, render=False):
    
    total_reward = []
    
    for game_round in range(10):
        
        game.reset_game()
        done = game.is_game_over()[0]
        
        while not done:
            action = int(np.argmax(Q[game.points]))
            game.make_move(action + 1)
            
            game.make_move(rplayer.move(game.points))
            done, turn, round_reward = game.is_game_over()
            
            if done:
                break
        
        total_reward.append(round_reward)        
    return np.mean(total_reward)

def q_learning(env, num_rounds, discount_factor=1.0, alpha=0.5, epsilon=0.1):

    Q = defaultdict(lambda: np.zeros(n))
    round_lengths = np.zeros(num_rounds)
    round_rewards = np.zeros(num_rounds)
    
    for game_round in range(num_rounds):
        
        game.reset_game()
        done = False
        
        while not done:
            state = game.points
            action = int(eps_greedy(Q, state, epsilon-epsilon/num_rounds*game_round))
            game.make_move(action + 1)
        
            round_lengths[game_round] += 1
            game.make_move(player.move(game.points))
            done, turn, reward = game.is_game_over()
                    
            round_rewards[game_round] += reward
            next_state = game.points
            
            td_target = reward + discount_factor * np.max(Q[next_state])
            td_error = td_target - Q[state][action]
            Q[state][action] += alpha * td_error
    
    return Q, round_lengths, round_rewards

In [64]:
Q, round_lengths, round_rewards = q_learning(game, 1000)
q_eval(Q)

1.0

# Задание 3. DQN (40%)

## Победите 10/10 используя DQN.
Подготовте алгоритм на основе Tabular Q-learning для 2го игрока.

Вы играете за 1го игрока, победите 2го.

In [106]:
class Q_learning_Player:
    """
    Player class
    """
    def move(self, points):
        """
        Player makes a move.
        """
        global Q
        action = int(np.argmax(Q[points]))
        return action + 1

In [107]:
qplayer = Q_learning_Player()

In [108]:
from collections import namedtuple
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import random
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [109]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [110]:
class DQN(nn.Module):
    
    def __init__(self, states, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=1, stride=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=1, stride=1)
        self.conv3 = nn.Conv1d(32, 32, kernel_size=1, stride=1)
        
        def conv2d_size_out(size, kernel_size = 1, stride = 1):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        
        linear_input_size = conv2d_size_out(conv2d_size_out(conv2d_size_out(states)))
        self.head = nn.Linear(32, outputs)

    def forward(self, x):
        x = x.reshape(-1, 1, 1)
        x = x.type(torch.float32)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return self.head(x.view(x.size(0), -1))

In [111]:
BATCH_SIZE = 16
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.005
EPS_DECAY = 200
TARGET_UPDATE = 10

n_states = 108
n_actions = 9

policy_net = DQN(n_states, n_actions).to(device)
target_net = DQN(n_states, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)

steps_done = 0
round_durations = []

In [112]:
def select_action(state, flag = False):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_START - EPS_START / num_rounds * (steps_done + 1)
    steps_done += 1
    if flag:
        with torch.no_grad():
            return policy_net(state).max(1)[1].view(1, 1)
    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)
    
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                        if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    
    state_action_values = policy_net(state_batch).gather(1, action_batch)
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
    
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.data.clamp_(-1, 1)
    optimizer.step()
    return

In [113]:
num_rounds = 1000

for game_round in range(num_rounds):
    steps_done = game_round
    game.reset_game()
    done = False
    
    for t in count():
        state = np.array([[game.points]], dtype=np.int)
        state = torch.from_numpy(state)

        state = state.unsqueeze(0).to(device).type(torch.float32)
        action = select_action(state)
        game.make_move(action + 1)
        game.make_move(qplayer.move(game.points))

        done, turn, reward = game.is_game_over()
        reward = torch.tensor([reward], device=device)

        if not done:
            next_state = np.array([[game.points]], dtype=np.int)
            next_state = torch.from_numpy(next_state)
            next_state = state.unsqueeze(0).to(device).type(torch.float32)
        else:
            next_state = None

        memory.push(state, action, next_state, reward)
        state = next_state
        optimize_model()
        if done:
            round_durations.append(t + 1)
            break
    if game_round % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

In [114]:
def dqn_eval(Q, render=False):
    total_reward = []
    
    for game_round in range(10):
        
        game.reset_game()
        done = False
        
        while not done:
            state = np.array([[game.points]], dtype=np.int)
            state = torch.from_numpy(state)
            state = state.unsqueeze(0).to(device).type(torch.float32)
            action = select_action(state, flag=True)
            
            game.make_move(action + 1)
            done, turn, round_reward = game.is_game_over()
            if done: break
                
            game.make_move(qplayer.move(game.points))
            done, turn, round_reward
            if done: break
                
        total_reward.append(round_reward)        
    return np.mean(total_reward)

In [115]:
dqn_eval(Q)

1.0

# Задание 4 (+2 балла при сделанном задании)
Опишите выигрышную стратегию для первого игрока.

Выигрышной стратегией, думаю, будет прибавление с целью получаения 90 на ходе игрока 1 <br>
То есть просто когда игра на ходе 1 игрока доходит до, к примеру, 87, он прибавляет именно 3, после этого 2 игрок прибавляя сколько угодно (от 1 до 9 в пределах правил) не получит больше 100, при это прибавление даже 1 позволит 1 игроку следующим шагом получить 100 <br>
Игра вроде простая, так что обошлась без латеха :)